#### Prerequisites

In [ ]:
%%capture 

!pip install transformers==4.18.0
!pip install datasets==2.4.0

### Imports 

In [2]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer
from transformers import pipeline
from datasets import load_metric 
import transformers
import pandas as pd
import datasets
import logging

In [3]:
pd.options.display.max_colwidth = 100

##### Setup logging

In [4]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [5]:
logger.info(f'[Using transformers: {transformers.__version__}]')
logger.info(f'[Using datasets: {datasets.__version__}]')

[Using transformers: 4.18.0]
[Using datasets: 2.4.0]


### Copy candidate models from S3 to local for evaluation 

In [ ]:
!aws s3 cp s3://sagemaker-us-east-1-119174016168/model/custom/ ./models/pretrained-from-scratch/ --recursive

### Load candidate models for evaluation 

##### Load Out-of-the-box (OOB) GPT2

In [6]:
oob_gpt2 = pipeline('text-generation', model='gpt2')

##### Load OOB GPT2 fine-tuned on covid news articles 

##### Load custom GPT2 further pre-trained (trained from scratch) on covid news articles 

In [7]:
custom_gpt2 = pipeline('text-generation', model='./models/pretrained-from-scratch/')

### Evaluate reference articles againsts the candidate models

In [8]:
ref_df = pd.read_csv('./data/test_articles.csv', names=['reference_article', 'prompt'])
ref_df

,reference_article,prompt
0,"On Tuesday, Dr. Fauci and other health officials testified before the U.S. House Energy and Comm...","On Tuesday, Dr. Fauci and other health officials"
1,Pfizer Inc. on Wednesday reported results from two late-stage studies ahead of schedule as it pu...,Pfizer Inc. on Wednesday reported results
2,President Donald Trump said the U.S. has the outbreak of the coronavirus under control and has b...,President Donald Trump said the U.S. has the outbreak
3,"President Joe Biden has signed a flurry of executive orders, actions and memorandums aimed at ra...",President Joe Biden has signed a flurry of executive orders
4,"China is effectively in a lockdown. From big cities to little villages, almost every community i...",China is effectively in a lockdown.
5,"Australian biotech Mesoblast has been riding high on expectations for its COVID-19 treatment, li...",Australian biotech Mesoblast has been riding high on expectations for its
6,"The December, 2019 coronavirus disease outbreak has seen many countries ask people who have pote...","The December, 2019 coronavirus disease outbreak has seen many countries ask people who have pote..."
7,The first confirmed case of coronavirus in India was reported today ( Jan. 30) in the southern s...,The first confirmed case of coronavirus in India was


In [9]:
ref_df['reference_article'][0]

'On Tuesday, Dr. Fauci and other health officials testified before the U.S. House Energy and Commerce Committee to discuss how the administration has been handling the coronavirus outbreak. Yahoo Finance’s Anjalee Khemlani breaks down the latest news about the coronavirus on The Final Round.'

In [10]:
for _, row in ref_df.iterrows():
    ref_article, prompt = row
    custom_gpt2_response = custom_gpt2(prompt, num_return_sequences=1, max_length=300, repetition_penalty=10.0, top_k=1, top_p=1.0)[0]['generated_text']
    oob_gpt2_response = oob_gpt2(prompt, num_return_sequences=1, max_length=300, repetition_penalty=10.0, top_k=1, top_p=1.0)[0]['generated_text']
    print(f'Prompt: {prompt}')
    print()
    print(f'Ref article: {ref_article}')
    print()
    print(f'Custom GPT2 Response: {custom_gpt2_response}')
    print()
    print(f'OOB GPT2 Response: {oob_gpt2_response}')
    print('-' * 200)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[2023-01-26 03:32:08.240 pytorch-1-8-gpu-py3-ml-g4dn-xlarge-60bd0d07a83be181dcf7335baae2:3606 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-01-26 03:32:08.273 pytorch-1-8-gpu-py3-ml-g4dn-xlarge-60bd0d07a83be181dcf7335baae2:3606 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: On Tuesday, Dr. Fauci and other health officials

Ref article: On Tuesday, Dr. Fauci and other health officials testified before the U.S. House Energy and Commerce Committee to discuss how the administration has been handling the coronavirus outbreak. Yahoo Finance’s Anjalee Khemlani breaks down the latest news about the coronavirus on The Final Round.

Custom GPT2 Response: On Tuesday, Dr. Fauci and other health officials in the city of tulsa have been working to develop a covid-19 testing site for residents who are asymptomatic or had mild symptoms but do not require hospitalization due directly by their employer ( e) - they will be tested at least once every two weeks starting on mondayth june 1st with results expected within 24 hours after that date; this is an important step towards reopening oklahoma’ s economy as it begins its phased reemergence from lockdown measures following months when cases were still low comparedwith many states across america where new infections 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Prompt: Australian biotech Mesoblast has been riding high on expectations for its

Ref article: Australian biotech Mesoblast has been riding high on expectations for its COVID-19 treatment, licensed to Novartis, but fell back to Earth after it said a phase 3 trial of the cell therapy was a bust. Shares in the stem cell specialist on the ASX lost more than a third of their value after data experts said the study of remestemcel-L in ventilator-dependent patients with moderate to severe acute respiratory distress syndrome ( ARDS) due to COVID-19 was unlikely to show a benefit. Mesoblast said the trial could have been affected by improvements in the care of COVID-19 patients over the last few months, as doctors gathered experience in treating the disease. That included the use of experimental drugs like dexamethasone and Gilead’ s antiviral Veklury (remdesivir).

Custom GPT2 Response: Australian biotech Mesoblast has been riding high on expectations for its covid-19 vaccine. the company’ s

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: The December, 2019 coronavirus disease outbreak has seen many countries ask people who have potentially come into contact with the infection to isolate themselves

Ref article: The December, 2019 coronavirus disease outbreak has seen many countries ask people who have potentially come into contact with the infection to isolate themselves at home or in a dedicated quarantine facility. Decisions on how to apply quarantine should be based on the best available evidence. This review of the psychological impact of quarantine using three electronic databases. Of 3166 papers found, 24 are included in this Review. Most reviewed studies reported negative psychological effects including post-traumatic stress symptoms, confusion, and anger. Stressors included longer quarantine duration, infection fears, frustration, boredom, inadequate supplies, inadequate information, financial loss, and stigma.

Custom GPT2 Response: The December, 2019 coronavirus disease outbreak has seen many countrie

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: The first confirmed case of coronavirus in India was

Ref article: The first confirmed case of coronavirus in India was reported today ( Jan. 30) in the southern state of Kerala. The patient, a female student at Wuhan University in China, tested positive for the novel coronavirus after returning to Kerala. Kerala health minister KK Shailaja has called an emergency meeting at 3pm.

Custom GPT2 Response: The first confirmed case of coronavirus in India was the second person to die from covid-19. he had been admitted on march 26th, and died at a hospital where his family lives with him after contracting it while working as an emergency room doctor for two years before being discharged last week ( april 1st).the death has prompted tributes across social media platforms including twitter twtr which said that mr cassidy’ s wife is now “ well enough ” but did not specify how many people were affected by their illness or who they believed might have contracted them.) one tweeter wrote: